In [2]:
from dataset import EEGDataset

dataset = EEGDataset('../data', limit=100, chunk_length=100, chunk_overlap=10, num_tokens=30)
print(dataset[0], len(dataset))

sample = dataset[0].unsqueeze(0).tile((3, 1, 1))
print(sample.shape)

torch.Size([4, 1693216])
tensor([[0.1111, 0.1118, 0.1118,  ..., 0.2179, 0.1334, 0.4000],
        [0.0760, 0.0769, 0.0768,  ..., 0.1184, 0.0548, 0.2657],
        [0.0395, 0.0405, 0.0400,  ..., 0.4000, 0.4000, 0.4000],
        [0.0685, 0.0702, 0.0696,  ..., 0.3511, 0.1597, 0.4000]]) 100
torch.Size([4, 1693216])
torch.Size([3, 4, 1693216])


In [3]:
sample = sample.unsqueeze(3)
sample.shape

torch.Size([3, 4, 1693216, 1])

In [5]:
import torch 

torch.nn.functional.unfold(sample, (250 * 2, 1), stride=(250 * 2, 1)).shape

torch.Size([3, 2000, 3386])

In [33]:
t = torch.arange(48, dtype=torch.float).reshape(1, 4, 12, 1)
torch.nn.functional.unfold(t, (4, 1), stride=(4, 1)).reshape(1, 4, 4, 3).transpose(1, 3).transpose(2, 3)

tensor([[[[ 0.,  1.,  2.,  3.],
          [12., 13., 14., 15.],
          [24., 25., 26., 27.],
          [36., 37., 38., 39.]],

         [[ 4.,  5.,  6.,  7.],
          [16., 17., 18., 19.],
          [28., 29., 30., 31.],
          [40., 41., 42., 43.]],

         [[ 8.,  9., 10., 11.],
          [20., 21., 22., 23.],
          [32., 33., 34., 35.],
          [44., 45., 46., 47.]]]])

In [29]:
t[0, :, :, 0]

tensor([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.],
        [12., 13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23.],
        [24., 25., 26., 27., 28., 29., 30., 31., 32., 33., 34., 35.],
        [36., 37., 38., 39., 40., 41., 42., 43., 44., 45., 46., 47.]])

In [52]:
def turn_into_patches(data, chunk_length, chunk_stride):
    assert len(data.shape) == 2 and data.shape[0] == 4, f'expected 2d tensor of shape (4, *), got {data.shape}'
    data_prep = data.unsqueeze(0).unsqueeze(3) # (1, 4, long, 1)
    chunked = torch.nn.functional.unfold(data_prep, kernel_size=(chunk_length, 1), stride=(chunk_stride, 1)) # (1, 4 * chunk_length, ~long // chunk_stride)
    assert len(chunked.shape) == 3 and chunked.shape[0] == 1 and chunked.shape[1] == 4 * chunk_length, f'expected (1, 4 * chunk_length, ~long // chunk_stride) got {chunked.shape}'
    chunked = chunked.reshape(1, 4, chunk_length, -1)
    print(chunked[0, :, :, 0].shape, data[:, :chunk_length].shape)
    assert (chunked[0, :, :, 0] == data[:, :chunk_length]).all(), "first chunk didn't match the beginning of data"
    return chunked
    
turn_into_patches(dataset[0], 500, 500).shape

torch.Size([4, 1693216])
torch.Size([4, 500]) torch.Size([4, 500])


torch.Size([1, 4, 500, 3386])

In [51]:
1693216 / 500

3386.432

In [43]:
dataset[0][:, :8]

torch.Size([4, 1693216])


tensor([[0.1111, 0.1118, 0.1118, 0.1118, 0.1123, 0.1113, 0.1118, 0.1117],
        [0.0760, 0.0769, 0.0768, 0.0765, 0.0769, 0.0762, 0.0769, 0.0768],
        [0.0395, 0.0405, 0.0400, 0.0391, 0.0394, 0.0397, 0.0409, 0.0402],
        [0.0685, 0.0702, 0.0696, 0.0680, 0.0683, 0.0685, 0.0702, 0.0696]])

In [4]:
from transformers import GPT2Config, GPT2Model

configuration = GPT2Config()
model = GPT2Model(configuration)

In [20]:
import torch

inp = torch.rand(8, 10, 768)
model.forward(inputs_embeds=inp, return_dict=True).last_hidden_state

odict_keys(['last_hidden_state', 'past_key_values'])